In [2]:
import xgboost
import joblib
import numpy as np
import pandas as pd

#### Load model, feature names, testing data and counterfactuals from testing set

In [3]:
model = joblib.load('files/model_asap1_776')
#model = joblib.load('files/model_asap7_776_counter_both_ori')

feature_names = joblib.load('files/feature_names_776')

data_test_ori = pd.read_csv('ASAP1 Test Set.tsv', sep='\t')

feat_test_ori = joblib.load('files/asap1_features_test_776')

feats_counter_test_776 = joblib.load('files/feats_counter_test_error_776')
np.array(feats_counter_test_776).shape

FileNotFoundError: [Errno 2] No such file or directory: 'files/model_asap1_776'

#### Predict the scores of the original essays

In [21]:
scores_ori = []
row_ctr = 0

for feat in feat_test_ori[:100]:
    dmat = xgboost.DMatrix(feat.reshape(1,-1), feature_names=feature_names)
    score = model.predict(dmat)
    scores_ori.append(score[0])
    row_ctr = row_ctr + 1
    print("Finished row number : ", row_ctr)

print("row number: ", row_ctr)

Finished row number :  1
Finished row number :  2
Finished row number :  3
Finished row number :  4
Finished row number :  5
Finished row number :  6
Finished row number :  7
Finished row number :  8
Finished row number :  9
Finished row number :  10
Finished row number :  11
Finished row number :  12
Finished row number :  13
Finished row number :  14
Finished row number :  15
Finished row number :  16
Finished row number :  17
Finished row number :  18
Finished row number :  19
Finished row number :  20
Finished row number :  21
Finished row number :  22
Finished row number :  23
Finished row number :  24
Finished row number :  25
Finished row number :  26
Finished row number :  27
Finished row number :  28
Finished row number :  29
Finished row number :  30
Finished row number :  31
Finished row number :  32
Finished row number :  33
Finished row number :  34
Finished row number :  35
Finished row number :  36
Finished row number :  37
Finished row number :  38
Finished row number :

In [5]:
scores_ori[:5]

[15.514046, 20.594347, 11.841169, 18.528923, 10.078994]

In [5]:
# joblib.dump(scores_ori, "scores_ori_test_776")
#scores_ori = joblib.load("scores_ori_test")

#### Predict the scores of the counterfactuals

In [22]:
scores_counter = []

row_ctr = 0

for row in feats_counter_test_776:
    row_score = []
    for essay in row:
        essay = xgboost.DMatrix(essay, feature_names=feature_names)
        score = model.predict(essay)
        row_score.append(score[0])
    scores_counter.append(row_score)
    row_ctr = row_ctr + 1
    print("Finished row number : ", row_ctr)

print("row number: ", row_ctr)

Finished row number :  1
Finished row number :  2
Finished row number :  3
Finished row number :  4
Finished row number :  5
Finished row number :  6
Finished row number :  7
Finished row number :  8
Finished row number :  9
Finished row number :  10
Finished row number :  11
Finished row number :  12
Finished row number :  13
Finished row number :  14
Finished row number :  15
Finished row number :  16
Finished row number :  17
Finished row number :  18
Finished row number :  19
Finished row number :  20
Finished row number :  21
Finished row number :  22
Finished row number :  23
Finished row number :  24
Finished row number :  25
Finished row number :  26
Finished row number :  27
Finished row number :  28
Finished row number :  29
Finished row number :  30
Finished row number :  31
Finished row number :  32
Finished row number :  33
Finished row number :  34
Finished row number :  35
Finished row number :  36
Finished row number :  37
Finished row number :  38
Finished row number :

In [8]:
joblib.dump(scores_counter, "files/scores_counterfactual_test_error_776")
#scores_counter = joblib.load('scores_counterfactual_test_error_776')

['files/scores_counterfactual_test_error_776']

In [9]:
np.array(scores_counter).shape

(100, 31)

#### Compare the scores Original vs Counterfactual

In [23]:
comparison = []

ctr = 1

for idx, row_scores in enumerate(scores_counter):
    ori_score = scores_ori[idx]
    compare_row = []
    for score in row_scores:

        if ctr <= 10:
            print(ctr)
            print("ori: ", ori_score)
            print("counter: ", score)
        
        score_diff = abs(score - ori_score)
        if score <= ori_score:
            compare_row.append((True, score_diff, score))
        else:
            compare_row.append((False, score_diff, score))
        ctr = ctr + 1
        
    comparison.append(compare_row)

1
ori:  13.265924
counter:  22.048935
2
ori:  13.265924
counter:  22.100119
3
ori:  13.265924
counter:  22.010702
4
ori:  13.265924
counter:  22.013065
5
ori:  13.265924
counter:  22.026842
6
ori:  13.265924
counter:  22.066175
7
ori:  13.265924
counter:  22.032795
8
ori:  13.265924
counter:  22.01203
9
ori:  13.265924
counter:  22.048935
10
ori:  13.265924
counter:  22.037153


In [25]:
comparison

[[(False, 8.7830105, 22.048935),
  (False, 8.834194, 22.100119),
  (False, 8.744778, 22.010702),
  (False, 8.747141, 22.013065),
  (False, 8.760918, 22.026842),
  (False, 8.800251, 22.066175),
  (False, 8.7668705, 22.032795),
  (False, 8.746105, 22.01203),
  (False, 8.7830105, 22.048935),
  (False, 8.771229, 22.037153),
  (False, 8.783522, 22.049446),
  (False, 8.844194, 22.110119),
  (False, 8.747141, 22.013065),
  (False, 8.744778, 22.010702),
  (False, 8.744778, 22.010702),
  (False, 8.731884, 21.997808),
  (False, 8.773762, 22.039686),
  (False, 8.849169, 22.115093),
  (False, 8.735291, 22.001215),
  (False, 8.7668705, 22.032795),
  (False, 8.7668705, 22.032795),
  (False, 8.737286, 22.00321),
  (False, 8.817465, 22.08339),
  (False, 8.783522, 22.049446),
  (False, 8.744778, 22.010702),
  (False, 8.727526, 21.99345),
  (False, 8.783167, 22.049091),
  (False, 8.784683, 22.050608),
  (False, 8.732927, 21.998852),
  (False, 8.779163, 22.045088),
  (False, 8.779291, 22.045216)],
 [(Fal

#### Count the number of False and mean of score difference for the 'False' predictions

In [26]:
# Initialize counters and lists to hold False values' second column values
false_count = 0
false_values = []
new_label = []
true_count = 0

# Iterate through each sublist
for idx, sublist in enumerate(comparison):
    ori = scores_ori[idx]
    row_label = []
    for value in sublist:

        formula = ori - value[1]
        
        if value[0] == False:  # If the first element is False
            false_count += 1
            false_values.append(value[1])  # Collect the second element (the difference)
            row_label.append(formula)
        else:
            true_count += 1
            row_label.append(value[2])
    new_label.append(row_label)
    
# Calculate the mean of the differences for False values only
if false_count > 0:
    mean_difference = np.mean(false_values)
else:
    mean_difference = 0  # Handle case where there are no False values

print(f"Count of False values: {false_count}")
print(f"Count of True values: {true_count}")
print(f"Mean of the differences for False values: {mean_difference}")


Count of False values: 1728
Count of True values: 1372
Mean of the differences for False values: 4.808871269226074


In [27]:
np.array(comparison).shape

(100, 31, 3)

In [28]:
acc = (true_count / 3100) * 100
acc

44.25806451612903

In [15]:
np.array(new_label).shape

(100, 31)

In [15]:
#joblib.dump(new_label, 'new_label_test_error')